# Submittin an FL Hub job

In [1]:
# add path
import sys
print(sys.executable)
print(sys.version)

NVFLARE_HOME=f"/home/hroth/Code2/nvflare/flhub"
HUB_EXAMPLE=f"{NVFLARE_HOME}/examples/flhub"

/home/hroth/Code2/nvflare/flhub/examples/flhub/nvflare_flhub/bin/python3.8
3.8.15 (default, Oct 12 2022, 19:15:16) 
[GCC 11.2.0]


In [2]:
# (optional) check environment
!python --version
!pip freeze | grep nvflare
!pip freeze | grep torch
!pip freeze | grep monai

Python 3.8.15
monai-nvflare==0.2.1
-e git+ssh://git@github.com/holgerroth/NVFlare.git@7569e5359161700fb215ae1ae90db263e54d636f#egg=nvflare_nightly
pytorch-ignite==0.4.10
torch==1.13.0
monai==1.0.1
monai-nvflare==0.2.1


### Open admin api runner

In [3]:
from nvflare.fuel.hci.client.fl_admin_api_runner import FLAdminAPIRunner, api_command_wrapper

# Initialize the runner
runner = FLAdminAPIRunner(
    username="admin@nvflare.com",
    admin_dir=f"{HUB_EXAMPLE}/workspaces/t1_workspace/admin@nvflare.com",
    poc=False,
    debug=False,
)

### Submit job

In [4]:
job_folder=f"{HUB_EXAMPLE}/job"
api_command_wrapper(runner.api.submit_job(job_folder))

{'status': <APIStatus.SUCCESS: 'SUCCESS'>, 'details': {'message': 'Submitted job: d23062a6-3171-4276-85e1-86fdea44915f', 'job_id': 'd23062a6-3171-4276-85e1-86fdea44915f'}, 'raw': {'time': '2022-11-09 18:29:15.570446', 'data': [{'type': 'string', 'data': 'Submitted job: d23062a6-3171-4276-85e1-86fdea44915f'}, {'type': 'success', 'data': ''}], 'status': <APIStatus.SUCCESS: 'SUCCESS'>}}


{'status': <APIStatus.SUCCESS: 'SUCCESS'>,
 'details': {'message': 'Submitted job: d23062a6-3171-4276-85e1-86fdea44915f',
  'job_id': 'd23062a6-3171-4276-85e1-86fdea44915f'},
 'raw': {'time': '2022-11-09 18:29:15.570446',
  'data': [{'type': 'string',
    'data': 'Submitted job: d23062a6-3171-4276-85e1-86fdea44915f'},
   {'type': 'success', 'data': ''}],
  'status': <APIStatus.SUCCESS: 'SUCCESS'>}}

### List jobs

In [5]:
%%capture
jobs = api_command_wrapper(runner.api.list_jobs())

In [6]:
print(jobs["raw"]["data"][0]["data"])

-------------------------------------------------------------------------------------------------------------------------------------------
| JOB ID                               | NAME          | STATUS                       | SUBMIT TIME                      | RUN DURATION   |
-------------------------------------------------------------------------------------------------------------------------------------------
| 5a2d5ac4-f882-49e5-8344-e011c3fa0967 | spleen-bundle | FINISHED:EXECUTION_EXCEPTION | 2022-11-09T17:51:58.701293-05:00 | 0:00:52.370493 |
| a4bd98bf-afa7-4162-8d12-21ee1324c507 | spleen-bundle | FINISHED:EXECUTION_EXCEPTION | 2022-11-09T18:23:06.803354-05:00 | 0:00:04.643823 |
| 572e49eb-e632-4056-94c6-312b95bfb8e1 | spleen-bundle | FINISHED:EXECUTION_EXCEPTION | 2022-11-09T18:26:12.399522-05:00 | 0:00:04.399591 |
| d23062a6-3171-4276-85e1-86fdea44915f | spleen-bundle | SUBMITTED                    | 2022-11-09T18:29:15.585640-05:00 | N/A            |
--------------------

### Show server log

In [7]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("server"))

In [8]:
print(reply["details"]["message"])

2022-11-09 18:26:13,778 - JobRunner - INFO - [identity=t1_project, run=?]: Got the job:572e49eb-e632-4056-94c6-312b95bfb8e1 from the scheduler to run
2022-11-09 18:26:13,796 - JobRunner - INFO - [identity=t1_project, run=?]: Application t1_app deployed to the server for job: 572e49eb-e632-4056-94c6-312b95bfb8e1
2022-11-09 18:26:13,805 - JobRunner - INFO - [identity=t1_project, run=?]: App t2a_app to be deployed to the clients: t1_client_a for run: 572e49eb-e632-4056-94c6-312b95bfb8e1
2022-11-09 18:26:13,812 - JobRunner - INFO - [identity=t1_project, run=?]: App t2b_app to be deployed to the clients: t1_client_b for run: 572e49eb-e632-4056-94c6-312b95bfb8e1
2022-11-09 18:26:14,754 - JobRunner - INFO - [identity=t1_project, run=?]: Started run: 572e49eb-e632-4056-94c6-312b95bfb8e1 for clients: t1_client_a,t1_client_b
2022-11-09 18:26:17,479 - FederatedServer - INFO - Could not connect to server runner process:  - asked client to end the run
2022-11-09 18:26:17,479 - FederatedServer - INF

### Show Connector-A log

In [9]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_a"))

In [10]:
print(reply["details"]["message"])

2022-11-09 18:23:10,821 - ProcessExecutor - INFO - run (a4bd98bf-afa7-4162-8d12-21ee1324c507): waiting for child worker process to finish.
2022-11-09 18:23:13,472 - ProcessExecutor - INFO - Client worker process is terminated.
2022-11-09 18:23:14,286 - ProcessExecutor - INFO - run (a4bd98bf-afa7-4162-8d12-21ee1324c507): child worker process finished with execution code: -9
2022-11-09 18:23:23,480 - ProcessExecutor - INFO - run (a4bd98bf-afa7-4162-8d12-21ee1324c507): child worker process terminated
2022-11-09 18:26:14,618 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-09 18:26:14,640 - ProcessExecutor - INFO - Worker child process ID: 385100
2022-11-09 18:26:16,645 - ProcessExecutor - INFO - run (572e49eb-e632-4056-94c6-312b95bfb8e1): waiting for child worker process to finish.
2022-11-09 18:26:19,096 - ProcessExecutor - INFO - Client worker process is terminated.
2022-11-09 18:26:20,156 - ProcessExecutor - INFO - run (572e49eb-e632-4056-94c6-312b95bfb8e1): child worker pr

### Show Connector-B log

In [11]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_b"))

In [12]:
print(reply["details"]["message"])

2022-11-09 18:23:10,857 - ProcessExecutor - INFO - run (a4bd98bf-afa7-4162-8d12-21ee1324c507): waiting for child worker process to finish.
2022-11-09 18:23:13,400 - ProcessExecutor - INFO - Client worker process is terminated.
2022-11-09 18:23:14,339 - ProcessExecutor - INFO - run (a4bd98bf-afa7-4162-8d12-21ee1324c507): child worker process finished with execution code: -9
2022-11-09 18:23:23,407 - ProcessExecutor - INFO - run (a4bd98bf-afa7-4162-8d12-21ee1324c507): child worker process terminated
2022-11-09 18:26:14,621 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-09 18:26:14,641 - ProcessExecutor - INFO - Worker child process ID: 385107
2022-11-09 18:26:16,647 - ProcessExecutor - INFO - run (572e49eb-e632-4056-94c6-312b95bfb8e1): waiting for child worker process to finish.
2022-11-09 18:26:19,092 - ProcessExecutor - INFO - Client worker process is terminated.
2022-11-09 18:26:20,090 - ProcessExecutor - INFO - run (572e49eb-e632-4056-94c6-312b95bfb8e1): child worker pr

### Finish

In [13]:
runner.api.logout()

{'status': <APIStatus.ERROR_INACTIVE_SESSION: 'ERROR_INACTIVE_SESSION'>,
 'details': 'session is logging out'}

# Show TensorBoard

In [14]:
# load only the hub server workspace
%load_ext tensorboard
%tensorboard --logdir "/home/hroth/Code2/nvflare/flhub/examples/flhub/workspaces/t1_workspace/localhost"

Note. The tensorboard should also be available at [http://localhost:6006](http://localhost:6006).